# Run pyBrep

pyBrep needs cloudpickle installed (additionally to what was already in the pybrep env that you set up for me)


In [30]:
% run ../BREPpy.py

## Specifications (the data that byBrep needs)

In [32]:
# Where the results will be stored
output_path = './output/d3_trial16/'

# Config file(s): see below
config_hoc = '../input_files/Parameters.hoc'
config_pseudo_hoc = 'pseudo_hoc.pkl'

# Coordinate input files
gol_in = '../example_simulation/coordinates_input/GoCcoordinates.sorted.dat'
gran_in ='../example_simulation/coordinates_input/GCcoordinates.sorted.dat'

## Config files
pyBREP originally uses a hoc object with the parameters necessary to run the simulation as configuration input.
However, if you want to run the simulation in an environment where neuron is not installed, you can do the following:
- run the following code in an environment which has neuron installed. It converts the hoc object to a dict and pickles it.
- Transfer the pickled file to the environment which does not have neuron installed and read it in as in the cell below the next.
- You get a preudo-hoc object which basically consists of it attributes, the parameters for the simulation. This oject can be used instead of the real hoc file.

In [23]:
ph = Pseudo_hoc()
ph.convert_hoc_to_pickle(config_hoc, config_pseudo_hoc)

In [26]:
# import config files
try: #if neuron is installed
    import neuron
    h = neuron.hoc.HocObject()
    neuron.h.xopen(config_hoc)
    print ('Trying to read in hoc config object from ', config_hoc)
except: 
    h = Pseudo_hoc(config_pseudo_hoc)
    print ('Trying to read in pseudo-hoc config object from ', config_pseudo_hoc)

Trying to read in hoc config object from  ../input_files/Parameters.hoc


## Population building

In [33]:
# Set up Golgi population, get dendrite points
gg = Golgi_pop(h)
gg.load_somata(gol_in)
gg.add_dendrites()

#Set up Granule population including aa and pf
gp = Granule_pop(h)
gp.load_somata(gran_in)
gp.add_aa_endpoints_fixed()
gp.add_aa_endpoints_random()
gp.add_pf_endpoints()


In [13]:
# Access the different points:

# soma coordinates
golgi_som = gg.som
#gran_som = gp.som

# The dendrites in a ncells x npoints x dim structure
print (gg.a_dend.shape)
print (gg.b_dend.shape)

# All dendrite points mashed together 
# (qpts is some other class that stores which cell the points belonged to)
print (gg.qpts.coo.shape)

#The parallel fiber and ascending axon endpoints: (ncells x [begin, end] x dim)
print (gp.aa_dots.shape)
print (gp.pf_dots.shape)


(1995, 50, 3)
(1995, 24, 3)
(147630, 3)
(798000, 2, 3)
(798000, 2, 3)


In [ ]:
# The rest should be done on the cluster...

## Subsampling

In [ ]:
# subsampling (constant density, smaller patch)

p1 = p_co #path to coordinate files
p2 = p_co #path where the subsampled data goes


#output filenames
fi_go = p1+'GoCcoordinates.dat'
fi_gr = p1+'GCcoordinates.dat'
fo_go64 = p2+'GoCcoordinates_64.dat'
fo_gr64 = p2+'GCcoordinates_64.dat'
fo_go16 = p2+'GoCcoordinates_16.dat'
fo_gr16 = p2+'GCcoordinates_16.dat'
fo_go4 = p2+'GoCcoordinates_4.dat'
fo_gr4 = p2+'GCcoordinates_4.dat' 


# Original size of membrane patch:
x_r = np.array([0.0, 1500.0])
y_r = np.array([0.0, 750.0])
z_r = [0.0, 1000.0]
# Smaller sizes
rrs64 = [x_r/8, y_r/8, z_r]
rrs16 = [x_r/4, y_r/4, z_r]
rrs4 = [x_r/2, y_r/2, z_r]


def subsample_coords (rrs, fn_in, fn_out = 'input_files/downsampled.dat', save = True):
    res = []
    rnr = [0, 0]
    with open(fn_in, newline = '') as f, open (fn_out, 'w', newline = '') as w_f:
        rr = csv.reader(f, delimiter = ' ')
        if save: wr = csv.writer(w_f, delimiter = ' ')
        for line in rr:
            in_range = all([float(line[i])>rrs[i][0] and float(line[i])<rrs[i][1] for i in range(len(rrs))]) #check if in range
            if in_range: 
                if save: wr.writerow([float(line[j]) for j in range(len(rrs))])
                res.append([float(line[j]) for j in range(len(rrs))])
                rnr[0] = rnr[0]+1
            else:
                rnr[1] = rnr[1]+1
    print ('Subsampled {} of {}'.format(rnr[0], sum(rnr)))
    return res




_ = subsample_coords (rrs64, fi_go, fo_go64)
_ = subsample_coords (rrs64, fi_gr, fo_gr64)

'''_ = subsample_coords (rrs16, fi_go, fo_go16)
_ = subsample_coords (rrs16, fi_gr, fo_gr16)
_ = subsample_coords (rrs4, fi_go, fo_go4)
_ = subsample_coords (rrs4, fi_gr, fo_gr4)
'''